# Terminologies

<img src="https://github.com/dorisjlee/remote/blob/master/astroSim-tutorial-img/terminology.jpg?raw=true",width=200,height=200>
- __Domain__ (aka Grids): the whole simulation box.
- __Block__(aka Zones): group of cells that make up a larger unit so that it is more easily handled. If the code is run in parallel, you could have one processor assigned to be in charge to work on several blocks (specified by iProcs,jProcs,kProcs in flash.par). In FLASH, the default block size in flash is $2^3$ = 8 cells. This means that level 0 in the AMR is 8 cells and so forth.

<img src="https://github.com/dorisjlee/remote/blob/master/astroSim-tutorial-img/level_cells.jpg?raw=true",width=200,height=200>
- __Cells__ : basic units that contain information about the fluid variables (often called primitives: $\rho$, $P$, $v_{x,y,z}$,$B_{x,y,z}$)
- __Ghost cells__ (abbrev as ``gc`` in FLASH): Could be thought of as an extra layer of padding outside the simulation domain. The alues of these gcs are mostly determined by what the boundary conditions you chose. Generally, you won't have to mess with these when specifying the initial conditions.




# Simulation_initBlock.F90

Simulation_initBlock is called by each block. First we compute the center based on the dimensions of the box (in cgs) from flash.par:

    center = abs(xmin-xmax)/2.

We loop through all the coordinates of the cell within each block. 

    do k = blkLimits(LOW,KAXIS),blkLimits(HIGH,KAXIS)
         ! get the coordinates of the cell center in the z-direction
         zz = zCoord(k)-center
         do j = blkLimits(LOW,JAXIS),blkLimits(HIGH,JAXIS)
            ! get the coordinates of the cell center in the y-direction
            yy = yCoord(j)-center
            do i = blkLimits(LOW,IAXIS),blkLimits(HIGH,IAXIS)
               ! get the cell center, left, and right positions in x
               xx  = xCenter(i)-center

``xCenter,yCoord,zCoord`` are functions that return the cell position (in cgs) given its cell index. These calculations are based on treating the bottom left corner of the box as the origin, so we minus the box center to get the origin to be at the center, as shown in Fig 3.

<img src="https://github.com/dorisjlee/remote/blob/master/astroSim-tutorial-img/user_coord.png?raw=true",width=200,height=200>
__Fig 3: The corrected ``xx,yy,zz`` are physical positions measured from the origin.__

Given the cell positions, you can specify values for initializing the fluid variables. 
The fluid variables are stored inside the local variables (called rhoZone,presZone,velxZone, velyZone,velzZone in the example) which are then transferred into to the cell one at a time using the method Grid_putData:

    call Grid_putPointData(blockId, CENTER, DENS_VAR, EXTERIOR, axis, rhoZone)
           
           
For example, you may have an analytical radial density distribution ($\rho= Ar^2$) that you would like to initialize the sphere with: 

    rr = sqrt(xx**2 + yy**2 + zz**2)
    rhoZone = A*rr**2

Or maybe your initial conditions can not be expressed in closed form,then you could also read in precomputed-values for each cell. This optional tutorial will explain how to do linear interpolation to setup the numerical solution of the Lane-Emden Sphere. 